### Set up a client to handle authentication etc with geoserver

In [5]:
import requests
import functools
from pprint import pprint
from IPython.core.display import display, HTML

BASE_URL = "http://127.0.0.1:8080/geoserver/rest"

class Client(object):
  def __init__(self, url, username='admin', password='geoserver'):
    self.base_url = url
    self.auth = {'auth': (username, password)}
    
  def _proxy(self, method, uri, *args, **kwargs):
    kwargs.update(self.auth)
    return method(self.base_url + uri, *args, **kwargs)

  def get(self, *args, **kwargs):
    return self._proxy(requests.get, *args, **kwargs)
  
  def put(self, *args, **kwargs):
    return self._proxy(requests.put, *args, **kwargs)
  
  def post(self, *args, **kwargs):
    return self._proxy(requests.post, *args, **kwargs)
  
  def delete(self, *args, **kwargs):
    return self._proxy(requests.delete, *args, **kwargs)
  
  def head(self, *args, **kwargs):
    return self._proxy(requests.head, *args, **kwargs)  
 
c = Client(BASE_URL)


### Make sure you have the geoserver VM running

this should show something like:

```
Current machine states:

geoserver                 running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.
```

You can do this by going to the geoserver/ directory and running ```vagrant up```

In [2]:
!cd ../geoserver && vagrant status

Current machine states:

geoserver                 running (virtualbox)

The VM is running. To stop this VM, you can run `vagrant halt` to
shut it down forcefully, or you can run `vagrant suspend` to simply
suspend the virtual machine. In either case, to restart it again,
simply run `vagrant up`.


### Display geoserver status

This should ensure the client can successfully connect to your VM,  if you do not see the Geoserver 'Status' page then something is wrong!

In [9]:
from IPython.core.display import display, HTML
display(HTML(c.get("/about/status").text))

### Create a workspace

In [6]:
c.post("/workspaces.json", 
       json={'workspace': {'name': 'test'}}) 

<Response [500]>

In [7]:
# Get the workspace
c.get("/workspaces/test.json").json()

{u'workspace': {u'coverageStores': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/coveragestores.json',
  u'dataStores': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/datastores.json',
  u'name': u'test',
  u'wmsStores': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/wmsstores.json'}}

### Get the rgb.tif file from data.kitware.com

In [19]:
!wget --output-file=/tmp/rgb.tif https://data.kitware.com/api/v1/file/57b471dd8d777f126827e6b7/download 

### Create coveragestore with the coverage and generate layers 

In [20]:
with open("/tmp/rgb.tif", 'rb') as fh:
  c.put("/workspaces/test/coveragestores/rgb/file.geotiff",
         params={"coverageName": 'rgb', 
                 "configure": "all"},
         headers={"Content-type": "image/tiff"},
         data=fh)

### Get the 'rgb' coverage

In [9]:
current = c.get("/workspaces/test/coveragestores/rgb/coverages/rgb.json").json()
current

{u'coverage': {u'defaultInterpolationMethod': u'nearest neighbor',
  u'description': u'Generated from GeoTIFF',
  u'dimensions': {u'coverageDimension': [{u'description': u'GridSampleDimension[-32768.0,-32768.0]',
     u'dimensionType': {u'name': u'UNSIGNED_8BITS'},
     u'name': u'RED_BAND',
     u'nullValues': {u'double': [-32768]},
     u'range': {u'max': -32768, u'min': -32768},
     u'unit': u'W.m-2.Sr-1'},
    {u'description': u'GridSampleDimension[-32768.0,-32768.0]',
     u'dimensionType': {u'name': u'UNSIGNED_8BITS'},
     u'name': u'GREEN_BAND',
     u'nullValues': {u'double': [-32768]},
     u'range': {u'max': -32768, u'min': -32768},
     u'unit': u'W.m-2.Sr-1'},
    {u'description': u'GridSampleDimension[-32768.0,-32768.0]',
     u'dimensionType': {u'name': u'UNSIGNED_8BITS'},
     u'name': u'BLUE_BAND',
     u'nullValues': {u'double': [-32768]},
     u'range': {u'max': -32768, u'min': -32768},
     u'unit': u'W.m-2.Sr-1'}]},
  u'enabled': True,
  u'grid': {u'@dimension': u

### Change some properties of the coverage

In [17]:
current['coverage']['dimensions']['coverageDimension'][0]['nullValues']['double'][0] = 0.0
current['coverage']['dimensions']['coverageDimension'][0]['range']['min'] = 1
current['coverage']['dimensions']['coverageDimension'][0]['range']['max'] = 255
current['coverage']['dimensions']['coverageDimension'][1]['nullValues']['double'][0] = 0.0
current['coverage']['dimensions']['coverageDimension'][1]['range']['min'] = 1
current['coverage']['dimensions']['coverageDimension'][1]['range']['max'] = 255
current['coverage']['dimensions']['coverageDimension'][2]['nullValues']['double'][0] = 0.0
current['coverage']['dimensions']['coverageDimension'][2]['range']['min'] = 1
current['coverage']['dimensions']['coverageDimension'][2]['range']['max'] = 255

c.put("/workspaces/test/coveragestores/rgb/coverages/rgb.json", json=current)

<Response [200]>

In [7]:
# Get the layer
# c.get("/layers/rgb.json").json()

{u'layer': {u'attribution': {u'logoHeight': 0, u'logoWidth': 0},
  u'defaultStyle': {u'href': u'http://127.0.0.1:8080/geoserver/rest/styles/raster.json',
   u'name': u'raster'},
  u'name': u'rgb',
  u'resource': {u'@class': u'coverage',
   u'href': u'http://127.0.0.1:8080/geoserver/rest/workspaces/test/coveragestores/rgb/coverages/rgb.json',
   u'name': u'rgb'},
  u'type': u'RASTER'}}

In [18]:
M.create_wms_layer("http://127.0.0.1:8080/geoserver/ows", "rgb")

In [3]:
M.region

BBox(ulx=-121.32263287061193, uly=47.81775513180975, lrx=-121.32144637512106, lry=47.81669282622947)

In [82]:
# Clean up,  delete the workspace
# c.delete("/workspaces/test")

<Response [200]>